In [1]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 18
plt.rcParams["font.family"] = "NanumGothicCoding"
mpl.rcParams["axes.unicode_minus"] = False
import seaborn as sns
%matplotlib inline

In [2]:
import folium

In [3]:
# 전국의 스타벅스 매장 찾기
# 1. requests로 위치 데이터 가져오기
# 2. json_normalize를 통해서 json 형태의 데이터를 데이터프레임을 변환
# 3. folium을 사용해서 지도위에 위치 표시

In [4]:
import requests
import json
from pandas.io.json import json_normalize

In [5]:
# 스타벅스 매장을 찾기 위한 요청 데이터를 작성한다.
# 위도와 경도는 데이터를 불러오는데 큰 의미는 없지만 지정하지 않으면 데이터를 불러오지 못한다.
postData = {
    "p_sido_cd": "01",               # 시도코드
    "p_gugun_cd": "",                # 구군코드
    "ins_lat": "37.5617072",         # 위도
    "ins_lng": "126.9751701",        # 경도 => 서울시청
    "in_biz_cd": "",
    "set_date": "",
    "iend": 2000                     # 최대로 얻어올 매장 정보의 개수
}
# 스타벅스 매장 정보를 얻어오는 url을 작성한다.
url = "https://www.starbucks.co.kr/store/getStore.do?r=GORF15813X"
# requests 모듈의 post() 메소드로 url과 요청 데이터를 넘겨서 매장 위치 정보를 얻어온다.
request = requests.post(url, data = postData)
print(type(request.text))
print(request.text)

<class 'str'>
{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1509","s_name":"역삼아레나빌딩","tel":"1522-3232","fax":"02-568-3763","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 역삼동 721-13 아레나빌딩","park_info":null,"new_state":null,"theme_state":"T04@T05@T08@T16@T17@T20@T21@T30@T36@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":"9.02","all_

In [6]:
# json 모듈의 loads() 메소드로 스타벅스 매장 위치 정보가 저장된 json 타입의 데이터를 파이썬에서
# 처리하기 위해 딕셔너리 타입으로 변환한다.
star_json = json.loads(request.text)
print(type(star_json))
print(star_json)

<class 'dict'>
{'list': [{'seq': 0, 'sido_cd': None, 'sido_nm': None, 'gugun_cd': None, 'gugun_nm': None, 'code_order': None, 'view_yn': None, 'store_num': None, 'sido': None, 'gugun': None, 'address': None, 'new_img_nm': None, 'p_pro_seq': 0, 'p_view_yn': None, 'p_sido_cd': '', 'p_gugun_cd': '', 'p_store_nm': None, 'p_theme_cd': None, 'p_wireless_yn': None, 'p_smoking_yn': None, 'p_book_yn': None, 'p_music_yn': None, 'p_terrace_yn': None, 'p_table_yn': None, 'p_takeout_yn': None, 'p_parking_yn': None, 'p_dollar_assent': None, 'p_card_recharge': None, 'p_subway_yn': None, 'stb_store_file_renew': None, 'stb_store_theme_renew': None, 'stb_store_time_renew': None, 'stb_store_lsm': None, 's_code': '1509', 's_name': '역삼아레나빌딩', 'tel': '1522-3232', 'fax': '02-568-3763', 'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 'addr': '서울특별시 강남구 역삼동 721-13 아레나빌딩', 'park_info': None, 'new_state': None, 'theme_state': 'T04@T05@T08@T16@T17@T20@T21@T30@T36@P80@P90', 'new_bo

In [7]:
# pandas.io.json 모듈의 json_normalize() 메소드로 json 타입의 데이터를 변환한 딕셔너리를 판다스
# 데이터프레임으로 변환한다.
star_df = json_normalize(star_json, "list")
print(type(star_df))
print(len(star_df))    # 행(row)의 개수, 서울에 2020년 7월 21일 현재 520개의 매장이 있다.
print(star_df.columns) # 열(column)의 개수 122개

<class 'pandas.core.frame.DataFrame'>
520
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p30', 'p70', 'p40', 'p80', 't22', 't21', 'p90', 't05', 't30', 't36'],
      dtype='object', length=122)


In [8]:
# 주요한 column 몇 가지를 선택해서 지도에 Marker를 표시할 때 사용할 데이터가 저장된 데이터프레임을
# 만든다.
# s_name       => 지점명
# tel          => 전화번호
# sido_code    => 시도코드
# sido_name    => 시도이름
# gugun_code   => 구군코드
# gugun_name   => 구군이름
# doro_address => 도로명 주소
# lat          => 위도
# lot          => 경도
star_df_map = star_df[["s_name", "lat", "lot", "sido_name", "gugun_name", "doro_address"]]
star_df_map

,s_name,lat,lot,sido_name,gugun_name,doro_address
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동)
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동)
2,신사역성일빌딩,37.514132,127.020563,서울,강남구,서울특별시 강남구 강남대로 584 (논현동)
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동)
4,스탈릿대치R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동)
...,...,...,...,...,...,...
515,중랑구청,37.60538908,127.0957558,서울,중랑구,서울특별시 중랑구 신내로 72
516,사가정역,37.579594,127.087966,서울,중랑구,서울특별시 중랑구 면목로 310
517,상봉역,37.59689,127.08647,서울,중랑구,"서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)"
518,묵동이마트,37.613433,127.077484,서울,중랑구,"서울특별시 중랑구 동일로 932, 묵동이마트 B1층 (묵동)"


In [9]:
# 위도, 경도 데이터의 타입을 object(문자열)에서 float(실수)로 변환한다.
star_df_map["lat"] = star_df_map["lat"].astype(float)
star_df_map["lot"] = star_df_map["lot"].astype(float)
star_df_map.dtypes

s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
dtype: object

In [10]:
# 종각점
star_df_map[star_df_map["s_name"] == "종각"]

,s_name,lat,lot,sido_name,gugun_name,doro_address
386,종각,37.569918,126.984528,서울,종로구,서울특별시 종로구 종로 64 (종로2가)


In [11]:
# 광교점
star_df_map[star_df_map["s_name"] == "광교"]

,s_name,lat,lot,sido_name,gugun_name,doro_address
387,광교,37.568928,126.984216,서울,종로구,서울특별시 종로구 청계천로 59 (관철동)


In [12]:
star_map = folium.Map(location = [37.566335, 126.9764516], zoom_start = 16)
folium.Marker(location = [37.566335, 126.9764516], popup = "서울특별시청").add_to(star_map)
folium.Marker(location = [37.569918, 126.984528], popup = "종각점").add_to(star_map)
folium.Marker(location = [37.568928, 126.984216], popup = "광교점").add_to(star_map)
star_map.save("./output/star1.html")
star_map

In [14]:
# 서울 스타벅스 전 지점
star_map = folium.Map(location = [37.566335, 126.9764516], zoom_start = 12)

# iterrows() 메소드는 판다스 데이터프레임의 인덱스 번호와 데이터프레임의 데이터를 한 행씩 리턴
# 시킨다.
for i, row in star_df_map.iterrows():
    folium.Marker(location = [row["lat"], row["lot"]], popup = row["s_name"]).add_to(star_map)

star_map.save("./output/star2.html")
star_map

In [34]:
# 종로구 스타벅스 전 지점
postData = {
    "p_sido_cd": "",
    "p_gugun_cd": "0118",
    "ins_lat": "37.5617072",
    "ins_lng": "126.9751701",
    "in_biz_cd": "",
    "set_date": "",
    "iend": 2000
}
url = "https://www.starbucks.co.kr/store/getStore.do?r=GORF15813X"
request = requests.post(url, data = postData)
star_json = json.loads(request.text)
star_df = json_normalize(star_json, "list")
star_df_map = star_df[["s_name", "lat", "lot", "sido_name", "gugun_name", "doro_address"]]
star_df_map

,s_name,lat,lot,sido_name,gugun_name,doro_address
0,동숭길입구,37.584236,127.002014,서울,종로구,서울특별시 종로구 대학로 146 (혜화동)
1,종로관수,37.569385,126.988665,서울,종로구,서울특별시 종로구 삼일대로20길 13 (관수동)
2,경복궁사거리,37.575578,126.980581,서울,종로구,"서울특별시 종로구 율곡로 6, B동 101호(중학동, 트윈트리타워)"
3,대학로,37.576509,127.001449,서울,종로구,"서울특별시 종로구 대학로 57, 홍익대학교 대학로캠퍼스 內 (연건동)"
4,종로3가,37.5705527,126.9902055,서울,종로구,"서울특별시 종로구 종로 113-1, 1-4층 (종로3가)"
5,마로니에공원,37.58154296,127.0023412,서울,종로구,"서울특별시 종로구 대학로 116, 1-2층 (동숭동)"
6,대명거리,37.58349633,127.0011593,서울,종로구,서울특별시 종로구 대명길 5 (명륜4가)
7,이마빌딩,37.5738637,126.97897149999994,서울,종로구,서울특별시 종로구 종로1길 42 (수송동)
8,경복궁역,37.5771508,126.97274530000004,서울,종로구,"서울특별시 종로구 자하문로4길 6, 1층 (통의동)"
9,종로관철,37.569058,126.986013,서울,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)"


In [35]:
star_map = folium.Map(location = [37.5711497, 126.9861408], zoom_start = 14)
for i, row in star_df_map.iterrows():
    folium.Marker(location = [row["lat"], row["lot"]], popup = row["s_name"]).add_to(star_map)
star_map.save("./output/star3.html")
star_map

In [27]:
# 제주도 스타벅스 전 지점
postData = {
    "p_sido_cd": "16",
    "p_gugun_cd": "",
    "ins_lat": "37.5617072",
    "ins_lng": "126.9751701",
    "in_biz_cd": "",
    "set_date": "",
    "iend": 2000
}
url = "https://www.starbucks.co.kr/store/getStore.do?r=GORF15813X"
request = requests.post(url, data = postData)
star_json = json.loads(request.text)
star_df = json_normalize(star_json, "list")
star_df_map = star_df[["s_name", "lat", "lot", "sido_name", "gugun_name", "doro_address"]]
star_df_map

,s_name,lat,lot,sido_name,gugun_name,doro_address
0,제주신화월드R,33.306608,126.316714,제주,서귀포시,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1-29
1,제주에듀시티,33.291788,126.284407,제주,서귀포시,제주특별자치도 서귀포시 대정읍 에듀시티로 36
2,제주성산DT,33.449980,126.920605,제주,서귀포시,제주특별자치도 서귀포시 성산읍 일출로 80
3,제주중문DT,33.2549983,126.41590589999998,제주,서귀포시,제주특별자치도 서귀포시 천제연로 95(색달동)
4,제주서귀포,33.24779669999999,126.5621036,제주,서귀포시,제주특별자치도 서귀포시 중정로 53 (서귀동)
5,제주송악산,33.206799,126.290035,제주,서귀포시,제주특별자치도 서귀포시 형제해안로 322 (상모리)
6,서귀포DT,33.2492775,126.50652289999993,제주,서귀포시,제주특별자치도 서귀포시 일주서로 11 (강정동)
7,성산일출봉,33.462865,126.935544,제주,서귀포시,"제주특별자치도 서귀포시 성산읍 일출로 284-5, 1,2층"
8,제주중문,33.251128,126.412752,제주,서귀포시,제주특별자치도 서귀포시 중문관광로110번길 32 (색달동)
9,제주도남DT,33.4803586,126.5234520,제주,제주시,제주특별자치도 제주시 연북로 394 (도남동)


In [32]:
star_map = folium.Map(location = [33.3616837, 126.5204118], zoom_start = 11)
for i, row in star_df_map.iterrows():
    folium.Marker(location = [row["lat"], row["lot"]], popup = row["s_name"]).add_to(star_map)
star_map.save("./output/star3.html")
star_map

In [37]:
# 경기도 양주 스타벅스 전 지점
postData = {
    "p_sido_cd": "",
    "p_gugun_cd": "0821",
    "ins_lat": "37.5617072",
    "ins_lng": "126.9751701",
    "in_biz_cd": "",
    "set_date": "",
    "iend": 2000
}
url = "https://www.starbucks.co.kr/store/getStore.do?r=GORF15813X"
request = requests.post(url, data = postData)
star_json = json.loads(request.text)
star_df = json_normalize(star_json, "list")
star_df_map = star_df[["s_name", "lat", "lot", "sido_name", "gugun_name", "doro_address"]]
star_df_map

,s_name,lat,lot,sido_name,gugun_name,doro_address
0,양주고읍,37.794628,127.079196,경기,양주시,경기도 양주시 고읍남로 20 (광사동)
1,양주광사DT,37.790532,127.072421,경기,양주시,경기도 양주시 부흥로 1872 (광사동)
2,양주고읍DT,37.803684,127.079474,경기,양주시,경기도 양주시 고덕로351 (고읍동)
3,양주덕정역DT,37.8382562,127.062968,경기,양주시,경기도 양주시 독바위로 35 (덕정동)
4,양주송추DT,37.717529,126.973093,경기,양주시,경기도 양주시 장흥면 호국로 528
5,양주옥정타운,37.818580,127.094293,경기,양주시,경기도 양주시 회천남로 76 (옥정동)


In [40]:
star_map = folium.Map(location = [37.7852912, 127.0436013], zoom_start = 12)
for i, row in star_df_map.iterrows():
    folium.Marker(location = [row["lat"], row["lot"]], popup = row["s_name"]).add_to(star_map)
star_map.save("./output/star3.html")
star_map